In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install ctransformers accelerate peft keybert jieba chinese-synonym-word

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 34.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39199 sha256=19fca65275b0315870a32aa8a25b02319090baa3f246b81a4b2df1d4ff58e632
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built keybert


**Load Llama2 for RAG**

In [5]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# Model
model_llama = AutoModelForCausalLM.from_pretrained(
    "SinpxAI/Llama2-Chinese-7B-Chat-GGUF",
    model_file="llama2-chinese-7b-chat.Q4_K_M.gguf",
    model_type="llama",
    gpu_layers=110,
    hf=True
)

# Tokenizer
tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=True)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ggml_cuda_set_main_device: using device 0 (Tesla T4) as main device


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
# Pipeline for Llama2
pipe_llama = pipeline(task="text-generation", model=model_llama, tokenizer=tokenizer_llama)

**Load Fine-tuned Bart Model & Tokenizer**

In [9]:
from transformers import BartForConditionalGeneration, AutoTokenizer, pipeline
from peft import PeftModel
import torch

base_model = "fnlp/bart-base-chinese"
new_model = "tonyma163/bart_v1"

device="cuda:0"

base_model_reload = BartForConditionalGeneration.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map=device,
        #trust_remote_code=True,
)
base_model_reload.half()

model_bart = PeftModel.from_pretrained(base_model_reload, new_model)

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/561M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.09M [00:00<?, ?B/s]

In [10]:
from transformers import BertTokenizer

tokenizer_bart = BertTokenizer.from_pretrained(base_model, trust_remote_code=True)

tokenizer_bart.pad_token = tokenizer_bart.eos_token
tokenizer_bart.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/259k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [11]:
from transformers import Text2TextGenerationPipeline

pipe_bart = Text2TextGenerationPipeline(model=model_bart, tokenizer=tokenizer_bart)

The model 'PeftModelForSeq2SeqLM' is not supported for . Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


**Load Knowledge Set**

In [13]:
import pandas as pd
import ast

file_path = "/kaggle/input/nlp-knowledge-set/knowledge_set.txt"

data = []

# Open the file and parse each line from string to tuple
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip():  # Ensure the line is not empty
            try:
                # Convert string representation of tuple to actual tuple
                tuple_data = ast.literal_eval(line.strip())
                data.append(tuple_data)
            except SyntaxError:
                print(f"Skipping malformed line: {line.strip()}")

# Load the data into a DataFrame
df = pd.DataFrame(data, columns=['Entity', 'Category', 'Answer'])

In [14]:
df.head()

,Entity,Category,Answer
0,西宁,2018-11-14,"阴,东风,最高气温:5℃,最低气温:-4℃"
1,何霄玲,喜好,poi
2,快乐大本营之快乐到家,评论,不好意思啊坡姐，我是你的路人黑，对不住了
3,辣相见川菜（三水总店）,特色菜,水煮鱼
4,浮城大亨,评论,人生是一幕大剧


**Knowledge Tree**

In [15]:
import networkx as nx
import pandas as pd

# Assuming 'df' is your DataFrame containing the data
G = nx.DiGraph()  # Directed graph can still function as a tree

# Add nodes and edges based on the DataFrame
for index, row in df.iterrows():
    entity_id = f"Entity: {row['Entity']}"
    category_id = f"Category: {row['Category']} ({row['Entity']})"
    answer_id = f"Answer: {row['Answer']} ({row['Category']})"

    # Ensure that nodes for each level (entity, category) are unique per entity-category pair
    if entity_id not in G:
        G.add_node(entity_id, type='Entity', name=row['Entity'])
    if category_id not in G:
        G.add_node(category_id, type='Category', name=row['Category'])
    
    # Answers can be multiple per category, so they are always added
    G.add_node(answer_id, type='Answer', content=row['Answer'])
    
    # Connect nodes hierarchically
    G.add_edge(entity_id, category_id)
    G.add_edge(category_id, answer_id)

**Part of Speech Function**

In [16]:
def checkSentence(sentence):
    separated_sentences = []
    temp_sentence = ""
    in_bracket = False

    for char in sentence:
        if char == "『" or char == "《":
            in_bracket = True
            if temp_sentence:
                separated_sentences.append(temp_sentence)
                temp_sentence = ""
            temp_sentence += char
        elif char == "』" or char == "》":
            in_bracket = False
            temp_sentence += char
            separated_sentences.append(temp_sentence)
            temp_sentence = ""
        elif char == "。" and not in_bracket:
            separated_sentences.append(temp_sentence)
            temp_sentence = ""
        else:
            temp_sentence += char

    if temp_sentence:
        separated_sentences.append(temp_sentence)

    return separated_sentences

In [17]:
import jieba.posseg as pseg

def extractPOS(sentences):

    result = []
    special_words = []

    for sentence in sentences:
        if '『' in sentence or '《' in sentence:
            # Remove the brackets
            sentence = sentence.replace('『', '@').replace('』', '@').replace('《', '@').replace('》', '@')
            #result.append(sentence)
            special_words.append(sentence)
        else:
            allowSentencePOS = ("a", "ad", "ag", "an", ## 形容词 
                                "b", # 区别词
                                "c", # 连词
                                "f", 
                                "g", 
                                "h",
                                "i", 
                                "j", 
                                "k", 
                                "l", 
                                "m", "mg", "mq", 
                                "n", "ng", "nr", "nrfg", "nrt","ns", "nt", "nz", 
                                "o", 
                                "q", 
                                "s", 
                                "t", "tg",
                                "v", "vd", "vg", "vi", "vn","vq" ## 动词
                                )

            exclude = ( "p", ## 介词
                "d", "df", "dg", # 副词
                "e", # 叹词 
                "r", "rg", "rr", "rz",  ## 代词
                "u",  "ud", "ug", "uj", "ul", "uv", "uz",  ## 助词
                "x",  ## 非语素词（包含标点符号
                "y", ## 语气词
                "z","zg" ## 助词
              )

            sentence = pseg.lcut(sentence)
            seg_words = [word for word, pos in sentence if pos in allowSentencePOS]
            result.extend(seg_words)
            
    return special_words+result

**Retrieve Data from Knowledge Tree**

In [20]:
def has_common_character(keyword, category):
    return any(char in category for char in keyword)

In [21]:
import jieba

def retrieve_answers(graph, query):
    segmented_keywords = extractPOS(checkSentence(query))
    special_keywords = [kw.strip('@') for kw in segmented_keywords if kw.startswith('@')]  # Process special keywords
    regular_keywords = [kw for kw in segmented_keywords if not kw.startswith('@')]  # Regular keywords

    found_answers = []

    # Top-Down Search: From entities to categories to answers with exact matching for entities
    for entity_node in (n for n in graph.nodes if graph.nodes[n]['type'] == 'Entity'):
        # Exact match required for entity names
        entity_keywords = [kw for kw in regular_keywords if kw == graph.nodes[entity_node]['name']]
        if entity_keywords:
            for category_node in graph.successors(entity_node):
                category_keywords = [kw for kw in regular_keywords if has_common_character(kw, graph.nodes[category_node]['name'])]
                if category_keywords:
                    for answer_node in graph.successors(category_node):
                        found_answers.append({
                            #graph.nodes[entity_node]['name']+
                            graph.nodes[category_node]['name']+
                            graph.nodes[answer_node]['content']
                        })
                        """found_answers.append({
                            'entity': graph.nodes[entity_node]['name'],
                            'category': graph.nodes[category_node]['name'],
                            'answer': graph.nodes[answer_node]['content']
                        })"""

    # Bottom-Up Search: From answers to categories to entities, but only using special keywords
    for answer_node in (n for n in graph.nodes if graph.nodes[n].get('type') == 'Answer'):
        if any(kw == graph.nodes[answer_node]['content'] for kw in special_keywords):  # Exact match check for special keywords
            for category_node in graph.predecessors(answer_node):
                if any(has_common_character(kw, graph.nodes[category_node]['name']) for kw in regular_keywords):
                    for entity_node in graph.predecessors(category_node):
                        found_answers.append({
                            graph.nodes[entity_node]['name']+
                            graph.nodes[category_node]['name']
                            #graph.nodes[answer_node]['content']
                        })
                        """found_answers.append({
                            'entity': graph.nodes[entity_node]['name'],
                            'category': graph.nodes[category_node]['name'],
                            'answer': graph.nodes[answer_node]['content']
                        })"""

    return found_answers

In [22]:
#Q1
input_query = "你知道张国荣的星座吗？"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q2
input_query = "周迅的星座是什么?"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q3
input_query = "范冰冰多重啊？"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q4
input_query = "『爱你等于爱自己』这首歌的主唱是谁啊？"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q5
input_query = "电影《倾城》主演是谁?"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q6
input_query = "说说张学友的新闻"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

Question: 你知道张国荣的星座吗？ Response: [{'星座处女座'}]
Question: 周迅的星座是什么? Response: [{'星座天秤座'}]
Question: 范冰冰多重啊？ Response: [{'体重56kg'}]
Question: 『爱你等于爱自己』这首歌的主唱是谁啊？ Response: [{'王力宏演唱'}]
Question: 电影《倾城》主演是谁? Response: [{'林心如主演'}]
Question: 说说张学友的新闻 Response: [{'新闻近日，江苏苏州，12月28日至30日张学友苏州演唱会再次上演“神助功”，三天时间里，苏州公安在全市范围内共抓获22名在逃人员。此前，苏州公安曾转发张学友演唱会苏州站预告，并称“懂了，懂了”。网友:公安战线著名歌手张学友要准备冲年度KPI了？'}, {'新闻距1/2世纪巡回演唱会已过去4年，今年，歌神再度回归，张学友2016,世界巡回演唱会北京站——“卡萨帝之夜”（张学友经典演唱会北京联合赞助）将于乐视体育中心开幕，呈现一场音乐与艺术的经典盛筵。那些年我们追过的歌神演唱会'}, {'新闻据四川遂宁警方介绍，21日晚张学友在遂宁第一场演唱会，警方挡获扒窃、卖假票、冒充工作人员骗钱骗票等涉嫌违法犯罪人员10余名。警方提醒:享受音乐盛宴的同时，也要提高警惕、谨防扒窃及各种骗局。（成都商报）?张学友演唱会又抓嫌犯！遂宁开唱第一场，警方就挡获10多人'}, {'新闻9月30日，陕西咸阳"逃犯克星"张学友举行演唱会时再次"立功"咸阳公安通报，不仅有5名逃犯落网，还抓获了其它违法犯罪人员13人9月28日晚，张学友演唱会在石家庄举行时便成功抓获了3名逃犯向娱乐圈卧底探员张Sir致敬！“逃犯克星”张学友又立功5名逃犯在咸阳落网'}, {'新闻据平安洛阳7月8日，民警在张学友演唱会现场附近执勤时发现一车辆交通违法，驾车男子提供个人信息不实，身份可疑。原来该男子从563公里远的武汉赶到洛阳，冒用他人信息无证驾驶，就是为看张学友演唱会。警察蜀黍棒棒哒(人民网)逃犯克星张学友上演第七杀？洛阳演唱会前一男子被抓'}, {'新闻7月8日，河南洛阳民警在张学友演唱会现场附近执勤时，发现一车辆存在交通违法，遂上前了解情况。经查，驾车男子提供的个人信息与其本人不符，身份可疑。经进一步核实，原来该男子

**Llama2(Knowledge Tree) + Fine-tuned Bart**

In [23]:
system_prompt = """
Please answer the following question based on the provided context.
Provide only the direct answers without any additional explanations or context.
Please only output answers only.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer, please don't share false information.
"""

In [24]:
def query_llama(query, context):
    # Test
    prompt = f"""
    <s>[INST] <<SYS>>
    {system_prompt}
     <</SYS>> [/INST]</s>

    <s>[INST]
    Question: {query}
    Context: {context}
    Answer:
    [/INST]
        """

    output = pipe_llama(
        prompt,
        do_sample=True,
        max_new_tokens=256,
        top_k=40,
        top_p=0.95,
        temperature=0.75,
        return_full_text=False,
        repetition_penalty=1
    )
    return output[0]['generated_text']

In [25]:
def query_system(graph, query):
    # Attempt to retrieve answers from the knowledge graph
    answers = retrieve_answers(graph, query)
    
    # If answers are found in the graph, return
    if answers:
        # pass to llama
        return "Answer from knowledge graph:", query_llama(query, answers)
    
    # If no answers are found, -> language model
    else:
        generated_answer = pipe_bart(query)
        return "Answer from language model:", generated_answer[0]['generated_text']

**Test**

In [26]:
query = "周迅的星座是什么？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '周迅的星座是天秤座。')


In [27]:
query = "你知道张柏芝的生日是什么时候吗？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '生日：1980-5-24\n')


In [28]:
query = "『爱你等于爱自己』这首歌的主唱是谁啊？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '王力宏')


In [29]:
query = "电影《倾城》主演是谁?"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '林心如主演电影《倾城》。')


In [30]:
query = "帮我点一首《想你》吧"
result = query_system(G, query)
print(result)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


('Answer from language model:', '正 在 为 你 播 放 《 想 你 》')


In [31]:
query = "张国荣哪里出生的？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '张国荣出生于香港九龙。\n')


In [32]:
query = "今天是什么日子了？"
result = query_system(G, query)
print(result)

('Answer from language model:', '今 天 是 2018 年 10 月 18 日')


In [33]:
query = "说说张学友的新闻"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', "\n    I don'这三个人民仴公安公安局长期��人��不提什苬告��。\n    I don'歌神助力控制定位于  * 张学友公安公安局面对于近日������������������������������������������������局长期��，] \n    I cannot provide only the news near  No direct answer: \n    Based on the newstories. 张学友网友网友消息，根据四川��了解����������通过去年����不知道��州公安局部分����������������������������到������������������所以上面对于22222222")
